In [1]:
import pyxdf
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns 
import wave
#import pyaudio
import numpy as np
#import sounddevice as sd
from utils import *
import cv2
from scipy.signal import iirnotch, filtfilt
from glob import glob

# Load Data

In [2]:
xdf_filename = '/Users/apurva.gokhe/Documents/CUNY_QC/data/sub-P5029423/sub-P5029423_ses-S001_task-CUNY_run-001_mobi.xdf'
stim_df = import_stim_data(xdf_filename)

# Stimulus

In [3]:
events = {
    200: 'Onset_ExperimentStart',
    10: 'Onset_RestingState',
    11: 'Offset_RestingState',
    500: 'Onset_StoryListening',
    501: 'Offset_StoryListening',
    100: 'Onset_10second_rest',
    101: 'Offset_10second_rest', 
    20: 'Onset_CampFriend',
    21: 'Offset_CampFriend',
    30: 'Onset_FrogDissection',
    31: 'Offset_FrogDissection',
    40: 'Onset_DanceContest',
    41: 'Offset_DanceContest',
    50: 'Onset_ZoomClass',
    51: 'Offset_ZoomClass',
    60: 'Onset_Tornado',
    61: 'Offset_Tornado',
    70: 'Onset_BirthdayParty',
    71: 'Offset_BirthdayParty',
    300: 'Onset_subjectInput',
    301: 'Offset_subjectInput',
    302: 'Onset_FavoriteStory',
    303: 'Offset_FavoriteStory',
    304: 'Onset_WorstStory',
    305: 'Offset_WorstStory',
    400: 'Onset_impedanceCheck',
    401: 'Offset_impedanceCheck',
    80: 'Onset_SocialTask',
    81: 'Offset_SocialTask',
    201: 'Offset_ExperimentEnd',
}


story_onsets = [20, 30, 40, 50, 60, 70]

In [11]:
audiofiles= [
    "/Users/apurva.gokhe/Documents/CUNY_QC/NEW_AUDIO_44/Camp_Lose_A_Friend.wav",
    "/Users/apurva.gokhe/Documents/CUNY_QC/NEW_AUDIO_44/Frog_Dissection_Disaster.wav",
    "/Users/apurva.gokhe/Documents/CUNY_QC/NEW_AUDIO_44/I_Decided_To_Be_Myself_And_Won_A_Dance_Contest.wav",
    "/Users/apurva.gokhe/Documents/CUNY_QC/NEW_AUDIO_44/I_Fully_Embarrassed_Myself_In_Zoom_Class1.wav",
    "/Users/apurva.gokhe/Documents/CUNY_QC/NEW_AUDIO_44/Left_Home_Alone_in_a_Tornado.wav",
    "/Users/apurva.gokhe/Documents/CUNY_QC/NEW_AUDIO_44/The_Birthday_Party_Prank_44.wav",
]

In [4]:
# Function to calculate time between two triggeres

def get_secs_between_triggers(trigger1, trigger2):
    return stim_df.loc[stim_df.trigger == trigger1, 'lsl_time_stamp'].values[0] - stim_df.loc[stim_df.trigger == trigger2, 'lsl_time_stamp'].values[0]

In [5]:
# confirm all events are in the stim_df
evs = stim_df.loc[stim_df.event != 'psychopy_time_stamp']
evs

,trigger,event,lsl_time_stamp,time
0,200.0,Onset_Experiment,185787.714273,0.000000
2,10.0,Onset_RestingState,185814.229936,26.515662
4,11.0,Offset_RestingState,186114.235558,326.521284
6,500.0,Onset_StoryListening,186114.235596,326.521322
8,100.0,Onset_10second_rest,186150.712077,362.997803
...,...,...,...,...
251,401.0,Offset_impedanceCheck,187676.164065,1888.449792
253,80.0,Onset_SocialTask,187693.919192,1906.204918
255,81.0,Offset_SocialTask,187994.410744,2206.696470
257,201.0,Offset_Experiment,187999.411616,2211.697343


### Check if markers are missing

In [6]:
def missing_markers(events, stim_df):
    missing_markers=[]
    for event in events:
        if event in stim_df.event:
            return None
        else:
            missing_markers = missing_markers + [event]
        return missing_markers

result = missing_markers(events, stim_df)
print(result)

None


### Checking durations of all story listening, resting state and social script

In [7]:
minutes_entire_experiment, seconds = divmod(get_secs_between_triggers(201, 200), 60) 
print('Duration of the entire experiment: ', f"{int(minutes_entire_experiment):02}:{int(seconds):02}")

Duration of the entire experiment:  36:51


In [13]:
# Get durations of all story listening tasks
durations = pd.DataFrame({
    'trigger':story_onsets,
    'story':[events[x] for x in story_onsets],
    'lsl_duration': [get_secs_between_triggers(x+1, x) for x in story_onsets],
    'audiofile_duration': [wave.open(x).getnframes()/wave.open(x).getframerate() for x in audiofiles] #duration of audio file is number of frames divided by the frame rate.
})
durations['difference(sec)'] = durations['audiofile_duration'] - durations['lsl_duration']
durations

,trigger,story,lsl_duration,audiofile_duration,difference(sec)
0,20,Onset_CampFriend,140.436595,152.838095,12.401500
1,30,Onset_FrogDissection,114.257204,124.344853,10.087649
2,40,Onset_DanceContest,127.101432,138.318730,11.217298
3,50,Onset_ZoomClass,90.384030,98.359864,7.975834
4,60,Onset_Tornado,149.983831,163.227619,13.243788
5,70,Onset_BirthdayParty,147.208981,160.215329,13.006348


### Check if story listening durations are within 500ms of their expected durations 

In [19]:
task_duration_diff = []

# Calculating audiofile duration in 48kHz and then comparing with story listening durations from stim_df
for i in range(len(durations.audiofile_duration)):
    task_duration = (durations.audiofile_duration[i] * 44100) / 48000
    if (durations.lsl_duration[i].round(3) -  task_duration.round(3)) > 0.5:
        task_duration_diff = task_duration_diff + [durations.story[i]]

if task_duration_diff != []:
    list_task_duration_diff = task_duration_diff
else:
    list_task_duration_diff = None

print('Durations do not match expected length: ', list_task_duration_diff)

Durations do not match expected length:  None


### Checking whether duration of resting state and social script is ~300s

In [15]:
def rest_and_social_duration(trigger):
    trial_duration = get_secs_between_triggers(trigger+1, trigger)
    #trial_duration = durations.loc[durations['trigger'] == trigger].lsl_duration.to_list()[0]
    print(trial_duration)
    if trial_duration <= 305.0  and trial_duration >= 298.0:
        return True
    else:
        return False

resting_state_duration_bool = rest_and_social_duration(10)
print('Is duration of resting state ~300s? ', resting_state_duration_bool)
social_script_duration_bool = rest_and_social_duration(80)
print('Is duration of social script ~300s? ', social_script_duration_bool)


300.00562166958116
Is duration of resting state ~300s?  True
300.4915517952759
Is duration of social script ~300s?  True


### Duration of Impedance check

In [16]:
#impedance_check_duration = durations.loc[durations.trigger == 400].lsl_duration.to_list()[0]
impedance_check_mins, impedance_check_seconds = divmod(get_secs_between_triggers(401, 400), 60)
print('Duration of Impedance check: ', f"{int(impedance_check_mins):02}:{int(impedance_check_seconds):02}")

Duration of Impedance check:  06:23


### Checking 10 s rest durations 

In [17]:
# Count the number of occurrences of trigger value 100 using sum
trigger_count = (evs['trigger'] == 100).sum()

print(f"Number of trigg: {trigger_count}")

#rest_onsets = 
ten_secs_rest_durations = pd.DataFrame({
    'trigger':[x for x in range(trigger_count)],
    'story': ['Onset_10second_rest' for x in range(trigger_count)],
    'lsl_duration': [get_secs_between_triggers(x+1, x) for x in evs['trigger'] if x == 100]})
#print(ten_secs_rest_durations)

# Check if rest durations are equal
equal_rest_durations = all(x == ten_secs_rest_durations['lsl_duration'][0] for x in ten_secs_rest_durations['lsl_duration'])
print("Are all 10 seconds rest equal?", equal_rest_durations)

Number of trigg: 6
Are all 10 seconds rest equal? True


### Average response time of questions across all trials

In [18]:
response_times = []
trial_response_times = []
trigger_idx = 0
for idx, x in enumerate(stim_df['trigger']):
    if idx in stim_df.loc[stim_df.trigger == 300,'lsl_time_stamp'].index:
        response_times = response_times + [stim_df.loc[stim_df.trigger == 301, 'lsl_time_stamp'].values[trigger_idx] - stim_df.loc[stim_df.trigger == 300, 'lsl_time_stamp'].values[trigger_idx]]
        trigger_idx = trigger_idx + 1
    elif x == 100 or x == 400:
        trial_response_times = trial_response_times + [sum(response_times)]
        response_times = []
average_response_mins, average_response_seconds = divmod(sum(trial_response_times)/len(trial_response_times), 60)     
print('Response time for all questions for each story listing task: ', trial_response_times) 
print('Average response time for all question across all trials: ', f"{int(average_response_mins):02}:{int(average_response_seconds):02}")  

Response time for all questions for each story listing task:  [0, 59.33065338549204, 34.60084509663284, 35.36828753026202, 31.26714915712364, 26.6756319811102, 37.603947086725384]
Average response time for all question across all trials:  00:32
